In [1]:
from pathlib import Path
from types import SimpleNamespace
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback

from data_loader import DataLoader
from utils.callbacks import TensorBoard
from utils.util import *
from utils.args_loader import load_model_config

2022-09-05 18:44:09.058099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-05 18:44:09.058136: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_path = Path("/mnt/disks/KITTI/small/")
data_path.exists()

True

In [3]:
# tf.config.run_functions_eagerly(True)

In [4]:
arg = SimpleNamespace(model="squeezesegv2",
                      config="squeezesegv2kitti",
                      data_path=data_path,
                      train_dir="../output",
                      epochs=10)   

config, model = load_model_config(arg.model, arg.config)
# config["DATA_AUGMENTATION"]=False

2022-09-05 18:44:11.284441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-05 18:44:11.285195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-05 18:44:11.285266: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-05 18:44:11.285328: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-09-05 18:44:11.285391: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [5]:
train_dl = DataLoader("train", arg.data_path, config).write_tfrecord_dataset().read_tfrecord_dataset()
val_dl = DataLoader("val", arg.data_path, config).write_tfrecord_dataset().read_tfrecord_dataset()

TFRecord exists at /mnt/disks/KITTI/small/train.tfrecord. Skipping TFRecord writing.
TFRecord exists at /mnt/disks/KITTI/small/val.tfrecord. Skipping TFRecord writing.


In [6]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=config.LEARNING_RATE,
    decay_steps=config.LR_DECAY_STEPS,
    decay_rate=config.LR_DECAY_FACTOR,
    staircase=True)

In [7]:
(lidar_input, lidar_mask), label, weight  = train_dl.take(1).get_single_element()

lidar_input.shape, lidar_mask.shape

(TensorShape([8, 64, 1024, 6]), TensorShape([8, 64, 1024]))

In [8]:
model.build([[8, 64, 1024, 6], [8, 64, 1024]])

ValueError: You cannot build your model by calling `build` if your layers do not support float type inputs. Instead, in order to instantiate and build your model, call your model on real tensor data (of the correct dtype).

The actual error from `call` is: Input 'condition' of 'SelectV2' Op has type float32 that does not match expected type of bool..

In [17]:
out = model([lidar_input, lidar_mask])

model.save("model.tf")

In [14]:
model.build([lidar_input.shape, lidar_mask.shape])

ValueError: You cannot build your model by calling `build` if your layers do not support float type inputs. Instead, in order to instantiate and build your model, call your model on real tensor data (of the correct dtype).

The actual error from `call` is: Input 'condition' of 'SelectV2' Op has type float32 that does not match expected type of bool..

## Let's log some data

We can use W&B to view the Dataset

In [7]:
def resize(img, factor=3):
    return tf.image.resize(img, [img.shape[0]*factor, img.shape[1]*factor]).numpy()

In [8]:
def log_input_data(lidar_input, label, class_color_map):
    "Log inputs to wandb"
    label = label[:, :]
    label_image = class_color_map[label.numpy().reshape(-1)].reshape([label.shape[0], label.shape[1], 3])
    depth_image = lidar_input.numpy()[:, :, [4]]
    intensity = lidar_input.numpy()[:, :, [3]]
    points = lidar_input.numpy()[:, :, :3]
    points_rgb = np.concatenate([points, (255*label_image).astype(int)], axis=-1).reshape(-1, 6)
    
    depth_image, label_image, intensity_image  = map(resize, [depth_image, label_image, intensity])
    
    # log 2 wandb
    wandb.log({'Images/Label Image': wandb.Image(label_image)})
    wandb.log({'Images/Depth Image': wandb.Image(depth_image)})
    wandb.log({'Images/Intensity Image': wandb.Image(intensity_image)})
    wandb.log({"Images/3D": wandb.Object3D({"type": "lidar/beta", "points":points_rgb})})

In [18]:
with wandb.init(project="small_kitti", entity="av-team", job_type="log_dataset"):
    (lidar_inputs, lidar_masks), labels, weights = val_dl.take(1).get_single_element() # a batch of 32 images
    for i, (lidar_input, label) in enumerate(zip(lidar_inputs, labels)):
        print(f"logging image: {i}")
        log_input_data(lidar_input, label, model.CLS_COLOR_MAP)

logging image: 0


In [36]:
import tensorflow as tf
import numpy


class LogSamplesCallback(WandbCallback):
    "A simple Keras callback to log model predictions"
    
    def __init__(self, dataset, **kwargs):
        super().__init__(**kwargs)
        self.dataset = dataset
        self.num_images = 1
        
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs=logs)
        
        # get first batch of dataset
        class_color_map = self.model.CLS_COLOR_MAP
        (lidar_input, lidar_mask), label, weight = self.dataset.take(1).get_single_element()

        probabilities, predictions = self.model([lidar_input, lidar_mask])
        predictions = predictions[:self.num_images, :, :].numpy()

        label = label[:self.num_images, :, :]
        

        # label and prediction visualizations
        label_image = class_color_map[label.numpy().reshape(-1)].reshape([self.num_images, label.shape[1], label.shape[2], 3])
        pred_image = class_color_map[predictions.reshape(-1)].reshape([self.num_images, label.shape[1], label.shape[2], 3])
        depth_image = lidar_input.numpy()[:self.num_images, :, :, [4]]
        intensity = lidar_input.numpy()[:self.num_images, :, :, [3]]
        points = lidar_input.numpy()[:self.num_images, :, :, :3]
        points_rgb = np.concatenate([points, (255*label_image).astype(int)], axis=-1).reshape(-1, 6)
        
        def _resize(img, factor=3):
            return tf.image.resize(img, [img.shape[1]*factor, img.shape[2]*factor]).numpy()
        intensity_image, depth_image, weight_image,label_image, pred_image = map(_resize, [intensity, depth_image, weight_image,label_image, pred_image])
        
        # log a bunch of images
        wandb.log({'Images/Label Image': wandb.Image(label_image)}, step=epoch)
        wandb.log({'Images/Depth Image': wandb.Image(depth_image)}, step=epoch)
        wandb.log({'Images/Intensity Image': wandb.Image(intensity_image)}, step=epoch)
        wandb.log({'Images/Prediction Image':wandb.Image(pred_image)}, step=epoch)
        wandb.log({"Images/3D": wandb.Object3D({"type": "lidar/beta", "points":points_rgb})})


In [21]:
wandb.init(project="small_kitti", entity="av-team", sync_tensorboard=True)

wandb_callback = WandbCallback(dataset=val_dl, save_model=False)
# tensorboard_callback = TensorBoard(arg.train_dir, val_dl, profile_batch=(200, 202))
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join(arg.train_dir, "checkpoint"))

# cbs = [wandb_callback, tensorboard_callback, checkpoint_callback]
cbs = [wandb_callback]

In [22]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=config.MAX_GRAD_NORM)

model.compile(optimizer=optimizer, weighted_metrics=[])

In [ ]:
model.fit(train_dl,
        validation_data=val_dl,
        epochs=arg.epochs,
        callbacks = cbs,
        )

wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/10


2022-09-05 14:57:00.897335: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1073741824 exceeds 10% of free system memory.
2022-09-05 14:57:01.457100: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 268435456 exceeds 10% of free system memory.
2022-09-05 14:57:01.639472: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1073741824 exceeds 10% of free system memory.
2022-09-05 14:57:02.602553: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100663296 exceeds 10% of free system memory.


      1/Unknown - 59s 59s/step - loss: 3.0744 - miou: 0.0546

In [ ]:
wandb.finish()